In [1]:
import pandas as pd

df = pd.read_csv(r'gen_dataset.csv')
text_data = list(df["text"])
class_data = df["class"].to_numpy()
for i in range(len(class_data)):
    if class_data[i]=='notsarc':
        class_data[i]=0
    else:
        class_data[i]=1
class_data = list(class_data)
train_sarc_text_data = text_data[:int(len(text_data) * 0.8)]
test_sarc_text_data = text_data[int(len(text_data) * 0.8):]

train_sarc_class_data = class_data[:int(len(class_data) * 0.8)]
test_sarc_class_data = class_data[int(len(class_data) * 0.8):]
df

,class,id,text
0,0,1,"If that's true, then Freedom of Speech is doom..."
1,0,2,Neener neener - is it time to go in from the p...
2,0,3,"Just like the plastic gun fear, the armour pie..."
3,0,4,So geology is a religion because we weren't he...
4,0,5,Well done Monty. Mark that up as your first ev...
...,...,...,...
6515,1,6516,depends on when the baby bird died. run alon...
6516,1,6517,"ok, sheesh, to clarify, women who arent aborti..."
6517,1,6518,so.. eh?? hows this sound? will it fly w...
6518,1,6519,"I think we should put to a vote, the right of ..."


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer
from torchmetrics.classification import BinaryAccuracy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load the pre-trained BERT model and tokenizer
model = AutoModel.from_pretrained('roberta-base')
model.to(device)
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits


# Define the training parameters
classifier = BertClassifier(model, num_classes=2)
classifier.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=2e-5)
metric = BinaryAccuracy().to(device)

cuda


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Load the training data
# train_texts = ["Example sentence 1", "Example sentence 2", ...]
# train_labels = [0, 1, ...]

# Tokenize the text data
train_encodings = tokenizer(list(train_sarc_text_data), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_sarc_text_data), padding=True, truncation=True, max_length=128)

# Convert the data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(train_sarc_class_data))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(list(test_sarc_class_data))
)


# Define the training parameters
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)
metric = BinaryAccuracy().to(device)


# Train the BERT classifier
classifier.train()
for epoch in range(10):
    print("Epoch ", epoch+1)
    train_losses = []
    train_acc = []
    test_losses = []
    test_acc = []
    for idx, batch in enumerate(train_loader):
        # batch.to(device)
        if(idx % 40 == 0):
            print("-", end="")
        optimizer.zero_grad()
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        loss.backward()
        train_losses.append(loss.cpu().detach().numpy())
        optimizer.step()
        op = torch.argmax(outputs, axis = 1)
        train_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
    for idx, batch in enumerate(test_loader):
        # batch.to(device)
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        test_losses.append(loss.cpu().detach().numpy())
        op = torch.argmax(outputs, axis = 1)
        test_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
        
    print()
    print("Train Loss: ",sum(train_losses) / len(train_losses), end = "\t")
    print("Test Loss: ",sum(test_losses) / len(test_losses), end = "\n")   
    print("Train Accuracy: ",(sum(train_acc) / len(train_acc)) * 100, end = "% \t")
    print("Test Accuracy: ",(sum(test_acc) / len(test_acc)) * 100, end = "% \n")   
    print()

Epoch  1
-----
Train Loss:  0.4648675854586385	Test Loss:  0.385839730137732
Train Accuracy:  76.61042944785275% 	Test Accuracy:  80.9451219512195% 

Epoch  2
-----
Train Loss:  0.3300090053695843	Test Loss:  0.44386409368456864
Train Accuracy:  84.91180981595092% 	Test Accuracy:  81.85975609756098% 

Epoch  3
-----
Train Loss:  0.24883697449338216	Test Loss:  0.3579204180618612
Train Accuracy:  89.99233128834356% 	Test Accuracy:  84.90853658536585% 

Epoch  4
-----
Train Loss:  0.15773754467490625	Test Loss:  0.5053636497113763
Train Accuracy:  93.90337423312883% 	Test Accuracy:  83.3079268292683% 

Epoch  5
-----
Train Loss:  0.09112624582074628	Test Loss:  0.5291093924968708
Train Accuracy:  96.54907975460122% 	Test Accuracy:  84.01930899154848% 

Epoch  6
-----
Train Loss:  0.05496251817072736	Test Loss:  0.6227445713267094
Train Accuracy:  97.89110429447852% 	Test Accuracy:  84.0701219512195% 

Epoch  7
-----
Train Loss:  0.040189075130152234	Test Loss:  0.7401736996522764
Train A

In [4]:
torch.save(classifier, "RoBERTa.pth")